# Labeling Cell Images
## Imports

In [ ]:
import cv2
import numpy as np
import os
from config import *
from CellProcessor import read_image, process_image, get_bboxes, get_label_yolo

## Make dir for labels

In [ ]:
LABELED_IMAGES_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Labeles_phase/"
IMAGES_MASKS_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Masks_phase/"
IMAGES_PHASE_PATH = CROP_PHASE_PATH
IMAGES_GREEN_PATH = CROP_GREEN_PATH

try:
    os.mkdir(LABELED_IMAGES_DIR_PATH)
except OSError:
    print("Dir already exists: " + LABELED_IMAGES_DIR_PATH)

try:
    os.mkdir(IMAGES_MASKS_DIR_PATH)
except OSError:
    print("Dir already exists: " + IMAGES_MASKS_DIR_PATH)

try:
    os.mkdir(IMAGES_PHASE_PATH)
except OSError:
    print("Dir already exists: " + IMAGES_PHASE_PATH)

try:
    os.mkdir(IMAGES_GREEN_PATH)
except OSError:
    print("Dir already exists: " + IMAGES_GREEN_PATH)


## For every image gen label

In [ ]:
images = os.listdir(GREEN_PATH)
len_images = len(images)
curr_image_num = 1

for image in images:
    print(str(curr_image_num) + "/" + str(len_images))
    curr_image_num+=1
    green_image_path = GREEN_PATH + image
    base_image_path = PHASE_PATH + image
    img, img_base = read_image(green_image_path, base_image_path)
    img_green = cv2.imread(green_image_path, cv2.IMREAD_UNCHANGED)

    img_base_h, img_base_w = img_base.shape[:2]

    CROP_WIDTH = 128
    CROP_HEIGHT = 128
    CROP_WIDTH_OVERLAP = 64
    CROP_HEIGHT_OVERLAP = 52
    CROP_WIDTH_SHIFT = CROP_WIDTH - CROP_WIDTH_OVERLAP
    CROP_HEIGHT_SHIFT = CROP_HEIGHT - CROP_HEIGHT_OVERLAP
    num_width_windows = (img_base_w - CROP_WIDTH)/CROP_WIDTH_SHIFT
    num_height_windows = (img_base_h - CROP_HEIGHT)/CROP_HEIGHT_SHIFT

    for width_window in range(int(num_width_windows)):
        for height_window in range(int(num_height_windows)):
            curr_topleft_x = width_window * CROP_WIDTH_SHIFT
            curr_topleft_y = height_window * CROP_HEIGHT_SHIFT
            img_crop = img[curr_topleft_y:curr_topleft_y + CROP_HEIGHT, curr_topleft_x:curr_topleft_x + CROP_WIDTH]
            img_base_crop = img_base[curr_topleft_y:curr_topleft_y + CROP_HEIGHT, curr_topleft_x:curr_topleft_x + CROP_WIDTH]
            img_green_crop = img_green[curr_topleft_y:curr_topleft_y + CROP_HEIGHT, curr_topleft_x:curr_topleft_x + CROP_WIDTH]

            img_dilation = process_image(img_crop, 
                                contrast, 
                                brightness, 
                                threshold, 
                                i_erode, 
                                i_dialate, 
                                plot = False)    

            bboxes = get_bboxes(img_dilation)
            for bbox in bboxes:
                for sp, ep in bbox:
                    if sp <0.0:
                        print(bbox)
                    if ep <0.0:
                        print(bbox)
            label_boxes = []
            mean_img = (np.sum(img_dilation)/4177920) * 100.0
            if len(bboxes) > 0 and mean_img < 40.0:
                for start_pt, end_pt in bboxes:
                    img_h, img_w = img_dilation.shape[:2]
                    x, y, w, h = get_label_yolo(start_pt, end_pt, img_w, img_h)
                    if w < 1.0 or h < 1.0:
                        label_box = "0 " + str(x) + " " + str(y) + " " + str(w) + " " + str(h) + "\n"
                        label_boxes.append(label_box)
                
                if label_boxes:
                    img_mask_file = IMAGES_MASKS_DIR_PATH + image.split(".")[0] + "_" + str(width_window) + "_" + str(height_window) + ".png"
                    img_file = IMAGES_PHASE_PATH + image.split(".")[0] + "_" + str(width_window) + "_" + str(height_window) + ".png"
                    img_green_file = IMAGES_GREEN_PATH + image.split(".")[0] + "_" + str(width_window) + "_" + str(height_window) + ".png"
                    cv2.imwrite(img_mask_file, img_dilation)
                    cv2.imwrite(img_file, img_base_crop)
                    cv2.imwrite(img_green_file, img_green_crop)
                    label_file = LABELED_IMAGES_DIR_PATH + "/" + image.split(".")[0] + "_" + str(width_window) + "_" + str(height_window) + ".txt"

                    with open(label_file, "w") as ann:
                        for lb in label_boxes:
                            ann.write(lb)